# Artificial Neural Networks and Deep Learning - Homework 1

<h3>Useful links</h3>

- [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)
- [keras_tuner](https://www.tensorflow.org/tutorials/keras/keras_tuner)
- [bayesian-optimization-in-cnn](https://www.kaggle.com/code/toniesteves/bayesian-optimization-in-cnn)
- [hyperparameter-search-with-bayesian-optimization-for-keras-cnn-classification-and-ensembling](https://machinelearningapplied.com/hyperparameter-search-with-bayesian-optimization-for-keras-cnn-classification-and-ensembling)
-[NN-SVG](https://alexlenail.me/NN-SVG)
- [tuning_playbook#contributing](https://github.com/google-research/tuning_playbook#contributing)


| Overfitters del test set    | PoliMi ID |
|:----------------------------|:---------:|
|Pertino Paolo                | 10729600  |
|Sandri Alberto               | 10698469  |
|Simionato Enrico             | 10698193  |

In the first homework of the Artificial Neural Networks and Deep Learning course, a binary image classification task is proposed. The goal is to understand from images of plants whether they are healthy or not.

## Table of contents

- [0. Preliminary operations](#0-preliminary-operations)
    - [0.1 Connect to drive](#01-connect-to-drive)
    - [0.2 Download and import libraries](#02-download-and-import-libraries)
- [1. Dataset inspection](#1-dataset-inspection)
    - [1.1 Loading the data](#11-loading-the-data)
    - [1.2 Outliers removal](#12-outliers-removal)

## 0. Preliminary operations

In the following section some preliminary operations in order to setup the environment correctly are performed.

### 0.1 Connect to Drive

In [ ]:
try:
    from google.colab import drive
    print("Code running on Google Colab... Conncecting to Google Drive...")
    drive.mount('/gdrive')
    %cd /gdrive/My Drive/[2023-2024] AN2DL/Homework 1
except:
    print("The code is not running on Google Colab...")

### 0.2 Download and import Libraries

In [ ]:
!pip install numpy
!pip install matplotlib
!pip install keras
!pip install tqdm
!pip install scikit-learn
!pip install tensorflow[and-cuda]
!pip install shap
!pip install pandas
!pip install scipy
!pip install Cython
!pip install opencv-python

In [ ]:
# Fix randomness and hide warnings
seed = 42

import os
import warnings
import logging
import random
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

random.seed(seed)

import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
#import shap
import cv2
np.random.seed(seed)

from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras.applications.mobilenet import preprocess_input
from tqdm import tqdm
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from IPython.display import Image
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)
print(f"Num GPUs Available: {len(tf.config.list_physical_devices('GPU'))}")

## 1. Dataset inspection

In this section the content of the dataset is inspected and the images are prepared for later analysis.

### 1.1 Loading the data

In [ ]:
data = np.load('data/public_data.npz', allow_pickle=True)

images = data.get('data')
labels = data.get('labels')

images.shape, labels.shape

As we can see, the dataset is composed by `5200` colored images of size `96x96`.

In [ ]:
images_normalized = (images / 255).astype(np.float32)

In [ ]:
# Number of images to display
num_img = 20

# Create subplots for displaying items
np.random.seed(312)
fig, axes = plt.subplots(2, num_img // 2, figsize=(20, 9))
for i in range(num_img):
    ax = axes[i % 2, i % num_img // 2]
    index_shown = np.random.randint(0, len(images))
    ax.imshow(np.clip(images_normalized[index_shown], 0, 255))  # Display clipped item images
    ax.axis('off')
    ax.set_title(f"{labels[index_shown]} - {index_shown}")
plt.tight_layout()
plt.show()

Hey! There are some outliers here! Images with index `4282` and `1903` are not plants :(

### 1.2 Outliers removal

Let's remove the outliers we have just found and all their copies in the dataset (if any).

In [ ]:
trololo = images[1903]
shrek = images[4282]

trolls = []
for i in tqdm(range(len(images))):
  if np.array_equiv(images[i], trololo) or np.array_equiv(images[i], shrek):
    trolls.append(i)

print(f"In the dataset there are {len(trolls)} outliers.")

In [ ]:
# Deleting the trolls
images_no_outliers = np.delete(images, trolls, axis = 0)
labels_no_outliers = np.delete(labels, trolls, axis = 0)

images_no_outliers.shape, labels_no_outliers.shape

### 1.3 Storing and loading the dataset without outliers

In [ ]:
# Storing the dataset without outliers
np.save('data/public_data_no_outliers', images_no_outliers, allow_pickle=True)
np.save('data/public_labels_no_outliers', labels_no_outliers, allow_pickle=True)

In [ ]:
images_no_outliers = np.load('data/public_data_no_outliers.npy', allow_pickle=True)
labels_no_outliers = np.load('data/public_labels_no_outliers.npy', allow_pickle=True)

images_no_outliers.shape, labels_no_outliers.shape

### 1.4 Plots and statistics

In [ ]:
# Defining the counts
counts = [sum(labels_no_outliers=='healthy'), sum(labels_no_outliers=='unhealthy')]

# Define labels
labels = ['healthy', 'unhealthy']

# Define colors for the bars
colors = ['lightgreen', 'salmon']

# Create the barplot
plt.bar(labels, counts, color=colors)

# Add axes labels
plt.xlabel('Classes - Health status')
plt.ylabel('Counts')

# Add title
plt.title('Class distribution')

# Add counts on top of the bars
for i, value in enumerate(counts):
  plt.text(labels[i], value, str(value), ha='center', va='bottom')

# Plot
plt.show()

In [ ]:
print(f"""
  In the healthy class there are {counts[0]} samples which cover the {(counts[0] / (counts[0] + counts[1]))*100:.2f}% of the whole dataset.
  In the unhealthy class there are {counts[1]} samples which cover the {(counts[1] / (counts[0] + counts[1]))*100:.2f}% of the whole dataset.
""")

The dataset is a little bit unbalanced towards the healthy class since there are 3101 samples in this class against 1903 of the other. However we think that this is not a big problem since the difference is not so big.

### 1.5 Pre-processing (2 classes)

In [ ]:
# Convert labels to one-hot encoding format
y = np.zeros((labels_no_outliers.shape[0], 2))
y[labels_no_outliers=='healthy', 0] = 1.
y[labels_no_outliers=='unhealthy', 1] = 1.

# Setting the input dataset to split
X = (images_no_outliers / 255).astype(np.float32)

# Defining the number of samples in the test set and in the validation set
validation_size = 750
test_size = 750

# Split data into train_val and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=seed, test_size=test_size, stratify=np.argmax(y,axis=1))

# Further split train_val into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=seed, test_size=validation_size, stratify=np.argmax(y_train_val,axis=1))

# Print shapes of the datasets
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

### 1.5.1 Pre-processing (1 class)

In [ ]:
# Convert labels to one-hot encoding format
y = np.zeros((labels_no_outliers.shape[0],))
y[labels_no_outliers=='healthy'] = 0.
y[labels_no_outliers=='unhealthy'] = 1.

# Setting the input dataset to split
X = (images_no_outliers / 255).astype(np.float32)

# Defining the number of samples in the test set and in the validation set
validation_size = 750  # 15%
test_size = 750  # 15%

# Split data into train_val and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=seed, test_size=test_size, stratify=y)

# Further split train_val into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=seed, test_size=validation_size, stratify=y_train_val)

# Print shapes of the datasets
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

### 1.6 Mixup Augmentation

In [ ]:
def mixup_images_labels(images : np.ndarray, labels : np.ndarray, alpha : float = 0.2, seed : int = 42):
    """
    Mixup augmentation for a dataset of images and labels.

    Parameters:
        images (numpy.ndarray): An array of images with shape (num_images, height, width, channels).
        labels (numpy.ndarray): An array of one-hot encoded labels with shape (num_images, num_classes).
        alpha (float): Mixup hyperparameter, controls the interpolation ratio.

    Returns:
        mixed_images (numpy.ndarray): Mixed images.
        mixed_labels (numpy.ndarray): Mixed labels.
    """
    num_images = images.shape[0]

    # Shuffle the dataset to ensure random mixing
    np.random.seed(seed)
    permutation = np.random.permutation(num_images)
    shuffled_images = images[permutation]
    shuffled_labels = labels[permutation]

    lam = np.random.beta(alpha, alpha, num_images)
    mixed_images = lam[:, np.newaxis, np.newaxis, np.newaxis] * images + (1 - lam[:, np.newaxis, np.newaxis, np.newaxis]) * shuffled_images
    mixed_labels = lam[:, np.newaxis] * labels + (1 - lam[:, np.newaxis]) * shuffled_labels

    return mixed_images, mixed_labels

X_train_mixup, y_train_mixup = mixup_images_labels(X_train, y_train, 0.15)

In [ ]:
# Number of images to display
num_img = 10

# Create subplots for displaying items
np.random.seed(312)
fig, axes = plt.subplots(2, num_img // 2, figsize=(20, 9))
for i in range(num_img):
    ax = axes[i % 2, i % num_img // 2]
    index_shown = np.random.randint(0, len(X_train_mixup))
    ax.imshow(np.clip(X_train_mixup[index_shown], 0, 255))  # Display clipped item images
    ax.axis('off')
    ax.set_title(f"{y_train_mixup[index_shown]} - {index_shown}")
plt.tight_layout()
plt.show()

### Fine tuning, blocks activation

In [ ]:
def activate_layers(model, name_layer, num_blocks_to_unfreeze, layers_to_skip=0, verbose=False):
  """
  Makes trainable a requested number of blocks of the given model

  """

  activated_last_layers = False

  model.get_layer(name_layer).trainable = False

  layers = ft_efficient_model.get_layer(name_layer).layers
  layers.reverse()

  skipped_layers = 0

  for i,layer in enumerate(layers):
    if 'block' in layer.name:
      if activated_last_layers:
        index_of_character = layer.name.index('_')
        current_block_name = layer.name[:index_of_character]

        if current_block_name != previous_block_name and skipped_layers>=layers_to_skip:
          num_blocks_to_unfreeze -= 1

        elif current_block_name != previous_block_name and skipped_layers<layers_to_skip:
          skipped_layers += 1
          print(skipped_layers)

        previous_block_name = current_block_name

      else:
        index_of_character = layer.name.index('_')
        previous_block_name = layer.name[:index_of_character]
        activated_last_layers = True

      if num_blocks_to_unfreeze > 0 and skipped_layers>=layers_to_skip:
        layer.trainable=True

    else:
      if activated_last_layers is False:
        layer.trainable=True

    if 'bn' in layer.name:
      layer.trainable = False

  if verbose:
    for i, layer in enumerate(ft_efficient_model.get_layer(name_layer).layers):
      print(i, layer.name, layer.trainable)

  return model

## 2. Models

### 2.1 Utility functions

In [ ]:
def plot_confusion_matrix(preprocessed_input : np.ndarray, y_test : np.ndarray, threshold : int = 0.5, model_to_test : tf.keras.Model = None, model_name : str = None, one_class : bool = False) -> None:
  '''
    Plots the confusion matrix and displays classification metrics for the
    specified model on the specified test data.

    The function first checks whether a model or a model name is provided for
    evaluation. If neither is provided, it will print an error message and
    return without further execution.

    If a model is provided (model_to_test is not None), the function uses it for
    evaluation. If only a model name is provided, the function loads the saved
    model with that name using tf.keras.models.load_model.

    It then predicts labels for the test data using the model and calculates
    the confusion matrix, as well as various classification metrics, including
    accuracy, precision, recall, and F1-score.

    The confusion matrix is plotted using a heatmap, and the classification
    metrics are displayed in the console.

    Parameters:
      preprocessed_input (numpy.ndarray): Preprocessed input data for testing.
      y_test (numpy.ndarray): Ground truth labels for the test data.
      model_to_test (tf.keras.Model, optional): The model to evaluate. If
      provided, this model will be used for evaluation.
      model_name (str, optional): The name of a saved Keras model to load for
      evaluation. If provided, this model will be loaded and used for
      evaluation.
      one_class (bool, optional): If True the output has one neuron. By default
      output has two neurons.

    Returns:
      None
  '''

  if model_to_test is None and model_name is None:
    print("You have to specify either a model name or pass directly the model you want to evaluate.")
    return

  if model_to_test is not None:
    # Loading the model
    model = model_to_test
  else:
    model = tf.keras.models.load_model(model_name)

  # Predict labels for the entire test set
  predictions_logits = model.predict(preprocessed_input, verbose=0)

  if one_class:
    predicted_label = np.array([1. if prediction >= threshold else 0. for prediction in predictions_logits])
    true_label = y_test
  else:
    predictions = np.array([[1., 0.] if sublist[0] >= threshold else [0., 1.] for sublist in predictions_logits])
    true_label = np.argmax(y_test, axis=-1)
    predicted_label = np.argmax(predictions, axis=-1)

  # Compute the confusion
  cm = confusion_matrix(true_label, predicted_label)

  # Compute classification metrics
  accuracy = accuracy_score(true_label, predicted_label)
  precision = precision_score(true_label, predicted_label)
  recall = recall_score(true_label, predicted_label)
  f1 = f1_score(true_label, predicted_label)

  # Display the computed metrics
  print('Accuracy:', accuracy.round(4))
  print('Precision:', precision.round(4))
  print('Recall:', recall.round(4))
  print('F1:', f1.round(4))

  # Plot the confusion matrix
  plt.figure(figsize=(10, 8))
  sns.heatmap(cm.T, xticklabels=list(('healthy','unhealthy')), yticklabels=list(('healthy','unhealthy')), cmap='Blues', annot=True)
  plt.xlabel('True labels')
  plt.ylabel('Predicted labels')
  plt.show()

  if model_name:
    del model

In [ ]:
def plot_roc_curve(model : tf.keras.Model, X : np.ndarray, y : np.ndarray, plot_diagrams : bool = True) -> float:
  '''
    Plots Receiver Operating Characteristic (ROC) curves for different threshold
    values and identifies the best threshold based on the highest AUC score.

    This function calculates and plots ROC curves for a given model's predictions
    on the input data, considering various threshold values (from 0.05 to 1 with
    step 0.05). It also identifies the best threshold based on the highest Area
    Under the Curve (AUC) score.

    Parameters:
        model (tf.keras.Model): The machine learning model for which the ROC
                                curves are plotted.
        X (numpy.ndarray): Input data used for model predictions.
        y (numpy.ndarray): Ground truth labels for the input data.

    Returns:
        float: the best threshold retrieved by the aalysis of the AUC score.

  '''

  roc_curves = []
  predictions_logits = model.predict(X)

  thresholds = np.arange(0.05, 1.05, 0.05)
  best_auc = 0
  best_threshold = 0

  for threshold in thresholds:
      # Convert predictions using the current threshold
      predictions = np.array([[1., 0.] if sublist[0] >= threshold else [0., 1.] for sublist in predictions_logits])

      # Calculate the ROC curve
      fpr, tpr, _ = roc_curve(y.argmax(axis=-1), predictions.argmax(axis=-1))

      roc_auc = auc(fpr, tpr)
      # Append the ROC curve data to the list
      roc_curves.append((fpr, tpr, threshold))

      if roc_auc > best_auc:
        best_auc = roc_auc
        best_threshold = threshold

  # Plot ROC curves
  if plot_diagrams:
    plt.figure(figsize=(8, 6))
    for fpr, tpr, threshold in roc_curves:
        plt.plot(fpr, tpr, label=f'Threshold: {threshold}')

    plt.plot([0, 1], [0, 1], 'k--', lw=2)  # Plot the diagonal line for reference
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate (FPR)')
    plt.ylabel('True Positive Rate (TPR)')
    plt.title('ROC Curve')
    plt.legend(loc='best')
    plt.show()
    print(f"The best threshold is {best_threshold} with AUC score of {best_auc}")
  return best_threshold

In [ ]:
def plot_training(history):
  '''
    Plots training and validation performance metrics, including loss and
    accuracy, and highlights the epoch with the highest validation accuracy.

    This function takes a history dictionary that contains training and
    validation metrics recorded during model training and visualizes the
    training and validation performance over epochs. It also highlights
    the epoch with the highest validation accuracy.

    Parameters:
        history (dict): A dictionary containing training and validation metrics
        (e.g., loss, accuracy) recorded during model training.

    Returns:
        None
  '''

  # Find the epoch with the highest validation accuracy
  best_epoch = np.argmax(history['val_accuracy'])

  # Plot training and validation performance metrics
  plt.figure(figsize=(20, 5))

  # Plot training and validation loss
  plt.plot(history['loss'], label='Training', alpha=0.8, color='#ff7f0e', linewidth=3)
  plt.plot(history['val_loss'], label='Validation', alpha=0.8, color='#4D61E2', linewidth=3)
  plt.legend(loc='upper left')
  plt.title('Binary Crossentropy')
  plt.grid(alpha=0.3)

  plt.figure(figsize=(20, 5))

  # Plot training and validation accuracy, highlighting the best epoch
  plt.plot(history['accuracy'], label='Training', alpha=0.8, color='#ff7f0e', linewidth=3)
  plt.plot(history['val_accuracy'], label='Validation', alpha=0.8, color='#4D61E2', linewidth=3)
  plt.plot(best_epoch, history['val_accuracy'][best_epoch], marker='*', alpha=0.8, markersize=10, color='#4D61E2')
  plt.legend(loc='upper left')
  plt.title('Accuracy')
  plt.grid(alpha=0.3)

  plt.show()

In [ ]:
def k_fold_cross_validation(
    train_data : np.ndarray,
    train_labels : np.ndarray,
    model_builder,
    model_builder_parameters : tuple = (),
    early_stopping_patience : int = 0,
    epochs : int = 200,
    batch_size : int = 32,
    model_callbacks : list = None,
    num_folds : int = 5,
    verbose = 0,
    seed : int = 42
  ) -> dict:
  '''
  Perform K-fold cross-validation for training and evaluating the specified
  model class.

  This function performs K-fold cross-validation by splitting the training
  data into multiple folds, training a model on each fold, and evaluating its
  performance on the validation data. It returns a dictionary with information
  about the training process for each fold.

  Parameters:
      train_data (numpy.ndarray): The training data used for cross-validation.
      train_labels (numpy.ndarray): The corresponding ground truth labels for the training data.
      model_builder (function): A function that builds and returns the model class of interest. This function should take model parameters as input.
      model_builder_parameters (tuple, optional): Additional parameters to be passed to the model_builder function.
      early_stopping_patience (int, optional): The number of epochs to wait for improvement during early stopping. Set this number equal to the one you have chosen for the early stopping.
      epochs (int, optional): The maximum number of epochs for training each fold.
      model_callbacks (list, optional): List of Keras callbacks to be applied during training.
      num_folds (int, optional): The number of cross-validation folds.
      seed (int, optional): Random seed for controlling the randomness of cross-validation.

  Returns:
      dict: A dictionary containing the training histories, validation scores, best epochs for early stopping, and best prediction thresholds for each fold.

  '''

  # Initialize lists to store training histories, scores, and best epochs
  result = {}
  histories = []
  scores = []
  best_epochs = []
  best_prediction_thresholds = []

  # Create a KFold cross-validation object
  kfold = KFold(n_splits=num_folds, shuffle=True, random_state=seed)

  # Loop through each fold
  for fold_idx, (train_idx, valid_idx) in enumerate(kfold.split(train_data, train_labels)):

    print(f"Starting training on fold num: {fold_idx+1}")

    # Build a new dropout model for each fold
    k_model = model_builder(*model_builder_parameters)

    # Train the model on the training data for this fold
    history = k_model.fit(
      x = X_train_val[train_idx],
      y = y_train_val[train_idx],
      validation_data=(X_train_val[valid_idx], y_train_val[valid_idx]),
      batch_size = batch_size,
      epochs = epochs,
      callbacks = model_callbacks,
      verbose = verbose
    ).history

    # Evaluate the model on the validation data for this fold
    score = k_model.evaluate(X_train_val[valid_idx], y_train_val[valid_idx], verbose=0)
    scores.append(score[1])

    # Calculate the best epoch for early stopping
    best_epoch = len(history['loss']) - early_stopping_patience
    best_epochs.append(best_epoch)

    # Store the training history for this fold
    histories.append(history)

    # Store the best prediction threshold
    best_prediction_thresholds.append(
        plot_roc_curve(
            k_model,
            X_train_val[valid_idx],
            y_train_val[valid_idx],
            plot_diagrams = False)
    )

  result.update({
      'histories' : histories,
      'scores' : scores,
      'best_epochs' : best_epochs,
      'best_thresholds' : best_prediction_thresholds
  })

  return result

In [ ]:
def undersample(X : np.ndarray, y : np.ndarray, one_class : bool = False) -> tuple:
  '''
    Perform undersampling on the given data.

    Parameters:
      X (numpy.ndarray): Preprocessed input data for training.
      y (numpy.ndarray): Ground truth labels for the train data.
      one_class (bool, optional): If True the output has one neuron. By default
      output has two neurons.

    Returns:
      tuple: A tuple containing the given X and y undersampled.
  '''

  reshaped_X = X.reshape(X.shape[0],-1)

  if not one_class:
    y = np.argmax(y, axis=-1)

  undersample = RandomUnderSampler()
  undersampled_X, undersampled_y  = undersample.fit_resample(reshaped_X , y)

  if not one_class:
    undersampled_y = np.array([[1., 0.] if sample == 0. else [0., 1.] for sample in undersampled_y])

  return undersampled_X.reshape(-1,96,96,3), undersampled_y

In [ ]:
def oversample(X : np.ndarray, y : np.ndarray, one_class : bool = False) -> tuple:
  '''
    Perform oversampling on the given data.

    Parameters:
      X (numpy.ndarray): Preprocessed input data for training.
      y (numpy.ndarray): Ground truth labels for the train data.
      one_class (bool, optional): If True the output has one neuron. By default
      output has two neurons.

    Returns:
      tuple: A tuple containing the given X and y overrsampled.
  '''

  reshaped_X = X.reshape(X.shape[0],-1)

  if not one_class:
    y = np.argmax(y, axis=-1)

  oversample = RandomOverSampler()
  oversampled_X, oversampled_y  = oversample.fit_resample(reshaped_X , y)

  if not one_class:
    oversampled_y = np.array([[1., 0.] if sample == 0. else [0., 1.] for sample in oversampled_y])

  return oversampled_X.reshape(-1,96,96,3), oversampled_y

### 2.2 Custom Baseline

In [ ]:
def build_custom_baseline(input_shape, output_shape, seed=seed):
  tf.random.set_seed(seed) # Execution on GPUs loses the deterministic execution guaranteed by the seed

  # Build the neural network layer by layer
  input_layer = tfkl.Input(shape=input_shape, name='Input')

  # Convolutional layers are linear layers
  conv1 = tfkl.Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='swish', name='conv1')(input_layer)

  #conv1_1 = tfkl.Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='swish', name='conv1_1')(conv1)

  pool1 = tfkl.MaxPooling2D(pool_size=(2,2), name='mp1')(conv1)

  conv2 = tfkl.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='swish', name='conv2')(pool1)

  #conv2_1 = tfkl.Conv2D(filters=32,kernel_size=(3,3), padding='same',activation='swish', name='conv2_1')(conv2)

  pool2 = tfkl.MaxPooling2D(pool_size =(2,2), name='mp2')(conv2)

  conv3 = tfkl.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='swish', name='conv3')(pool2)

  #conv3_1 = tfkl.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='swish', name='conv3_1')(conv3)

  #pool3 = tfkl.MaxPooling2D(pool_size=(2,2), name='mp3')(conv3)

  #conv4 = tfkl.Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='swish', name='conv4')(pool3)

  #conv4_1 = tfkl.Conv2D(filters=128, kernel_size=(3,3),padding='same',activation='swish', name='conv4_1')(conv4)

  gap = tfkl.GlobalAveragePooling2D(name='gap')(conv3)

  classifier_layer1 = tfkl.Dense(
      units=120,
      activation='swish',
      name='dense1'
  )(gap)

  classifier_layer2 = tfkl.Dense(
      units=84,
      activation='swish',
      name='dense2'
  )(classifier_layer1)

  output_layer = tfkl.Dense(
      units=output_shape,
      activation='softmax',
      name='Output'
  )(classifier_layer2)

  # Connect input and output through the Model class
  model = tfk.Model(inputs=input_layer, outputs=output_layer, name='custom_baseline')

  # Compile the model
  model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.AdamW(), metrics=['accuracy'])

  # Return the model
  return model

In [ ]:
input_shape = (96, 96, 3)
output_shape = 2

# Build the LeNet model and display its summary
model = build_custom_baseline(input_shape, output_shape)
model.summary()

In [ ]:
#dropout = tfkl.Dropout(dropout_rate, seed=seed)(hidden_activation)

In [ ]:
# Define the hyperparameters
batch_size = 16
epochs = 200

# Define early stopping callback
early_stopping = tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, mode='max', restore_best_weights=True)
lr_patience = 5
lr_scheduler = tfk.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',     # Metric to monitor (validation mean squared error in this case)
    patience=lr_patience,  # Number of epochs with no improvement after which learning rate will be reduced
    factor=0.999,          # Factor by which the learning rate will be reduced (0.999 in this case)
    mode='max',            # Mode to decide when to reduce learning rate ('min' means reduce when metric stops decreasing)
    min_lr=1e-5            # Minimum learning rate
)

# Train the model and save its history
history = model.fit(
    x=X_train,
    y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, lr_scheduler]
).history

# Save the trained model
model.save('models/custom_baseline_multiple_conv')

In [ ]:
# Find the epoch with the highest validation accuracy
best_epoch = np.argmax(history['val_accuracy'])

# Plot training and validation performance metrics
plt.figure(figsize=(20, 5))

# Plot training and validation loss
plt.plot(history['loss'], label='Training', alpha=0.8, color='#ff7f0e', linewidth=3)
plt.plot(history['val_loss'], label='Validation', alpha=0.8, color='#4D61E2', linewidth=3)
plt.legend(loc='upper left')
plt.title('Binary Crossentropy')
plt.grid(alpha=0.3)

plt.figure(figsize=(20, 5))

# Plot training and validation accuracy, highlighting the best epoch
plt.plot(history['accuracy'], label='Training', alpha=0.8, color='#ff7f0e', linewidth=3)
plt.plot(history['val_accuracy'], label='Validation', alpha=0.8, color='#4D61E2', linewidth=3)
plt.plot(best_epoch, history['val_accuracy'][best_epoch], marker='*', alpha=0.8, markersize=10, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=0.3)

plt.show()

In [ ]:
plot_confusion_matrix('models/custom_baseline_multiple_conv', X_test, y_test)

#### Training again on the whole dataset

In [ ]:
# Define the test size
test_size = 100

# Split data into train_val and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed, test_size=test_size, stratify=np.argmax(y,axis=1))

In [ ]:
input_shape = (96, 96, 3)
output_shape = 2

# Build the LeNet model and display its summary
model = build_custom_baseline(input_shape, output_shape)
model.summary()

In [ ]:
# Define the hyperparameters
batch_size = 16
epochs = 200

# Define early stopping callback
early_stopping = tfk.callbacks.EarlyStopping(monitor='accuracy', patience=20, mode='max', restore_best_weights=True)

# Define learning rate scheduler callback
lr_patience = 5
lr_scheduler = tfk.callbacks.ReduceLROnPlateau(monitor='accuracy', patience=lr_patience, factor=0.999, mode='max', min_lr=1e-5)

# Train the model and save its history
history = model.fit(
    x=X,
    y=y,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[early_stopping, lr_scheduler]
).history

# Save the trained model
model.save('models/custom_baseline_multiple_conv_whole_dataset')

In [ ]:
# Save the trained model
model.save('models/custom_baseline_multiple_conv_whole_dataset')

In [ ]:
# Find the epoch with the highest validation accuracy
best_epoch = np.argmax(history['accuracy'])

# Plot training and validation performance metrics
plt.figure(figsize=(20, 5))

# Plot training and validation loss
plt.plot(history['loss'], label='Training', alpha=0.8, color='#ff7f0e', linewidth=3)
plt.legend(loc='upper left')
plt.title('Binary Crossentropy')
plt.grid(alpha=0.3)

plt.figure(figsize=(20, 5))

# Plot training and validation accuracy, highlighting the best epoch
plt.plot(history['accuracy'], label='Training', alpha=0.8, color='#ff7f0e', linewidth=3)
plt.plot(best_epoch, history['accuracy'][best_epoch], marker='*', alpha=0.8, markersize=10, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=0.3)

plt.show()

In [ ]:
plot_confusion_matrix('models/custom_baseline_multiple_conv_whole_dataset', X_test, y_test)

In [ ]:
# Find the epoch with the highest validation accuracy
best_epoch = np.argmax(history['val_accuracy'])

# Plot training and validation performance metrics
plt.figure(figsize=(20, 5))

# Plot training and validation loss
plt.plot(history['loss'], label='Training', alpha=0.8, color='#ff7f0e', linewidth=3)
plt.plot(history['val_loss'], label='Validation', alpha=0.8, color='#4D61E2', linewidth=3)
plt.legend(loc='upper left')
plt.title('Binary Crossentropy')
plt.grid(alpha=0.3)

plt.figure(figsize=(20, 5))

# Plot training and validation accuracy, highlighting the best epoch
plt.plot(history['accuracy'], label='Training', alpha=0.8, color='#ff7f0e', linewidth=3)
plt.plot(history['val_accuracy'], label='Validation', alpha=0.8, color='#4D61E2', linewidth=3)
plt.plot(best_epoch, history['val_accuracy'][best_epoch], marker='*', alpha=0.8, markersize=10, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=0.3)

plt.show()

### 2.2 Paolo's models

### 2.3 Alberto's models

#### Custom model

In [ ]:
def build_custom_baseline(input_shape, output_shape, seed=seed):
  tf.random.set_seed(seed)
  dropout_rate = 1/6

  preprocessing = tfk.Sequential([
        tfkl.RandomFlip('horizontal'),
        tfkl.RandomFlip('vertical'),
        tfkl.RandomRotation(0.2),
        tfkl.RandomZoom(0.2),
    ], name='Preprocessing')

  # Build the neural network layer by layer
  input_layer = tfkl.Input(shape=input_shape, name='Input')

  preprocessing = preprocessing(input_layer)

  # Convolutional layers are linear layers
  x = tfkl.Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', name='conv1')(preprocessing)
  x = tfkl.Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', name='conv1_1')(x)
  x = tfkl.BatchNormalization(name='batchNorm1')(x)
  x = tfkl.MaxPooling2D(pool_size=(2,2), name='mp1')(x)
  x = tfkl.Dropout(dropout_rate, seed=seed)(x)

  x = tfkl.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', name='conv2')(x)
  x = tfkl.Conv2D(filters=64,kernel_size=(3,3), padding='same',activation='relu', name='conv2_1')(x)
  x = tfkl.BatchNormalization(name='batchNorm2')(x)
  x = tfkl.MaxPooling2D(pool_size =(2,2), name='mp2')(x)
  x = tfkl.Dropout(dropout_rate, seed=seed)(x)

  x = tfkl.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu', name='conv3')(x)
  x = tfkl.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu', name='conv3_1')(x)
  x = tfkl.BatchNormalization(name='batchNorm3')(x)
  x = tfkl.MaxPooling2D(pool_size=(2,2), name='mp3')(x)
  x = tfkl.Dropout(dropout_rate, seed=seed)(x)

  x = tfkl.Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu', name='conv4')(x)
  x = tfkl.Conv2D(filters=256, kernel_size=(3,3),padding='same',activation='relu', name='conv4_1')(x)
  x = tfkl.BatchNormalization(name='batchNorm4')(x)
  x = tfkl.GlobalAveragePooling2D(name='gap')(x)
  x = tfkl.Dropout(dropout_rate, seed=seed)(x)

  output_layer = tfkl.Dense(units=2, activation='softmax',name='Output')(x)

  #classifier_layer2 = tfkl.Dense(units=84, activation='relu', name='dense2')(classifier_layer1)
  #output_layer = tfkl.Dense(units=output_shape, activation='softmax', name='Output')(classifier_layer2)

  # Connect input and output through the Model class
  model = tfk.Model(inputs=input_layer, outputs=output_layer, name='custom_baseline')

  # Compile the model
  model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.AdamW(), metrics=['accuracy'])

  # Return the model
  return model

In [ ]:
from IPython.display import Image

image = tfk.utils.plot_model(model, show_shapes=True, expand_nested=True)

# Define the output file path
output_file = "models/custom_baseline4/model_image.png"

# Save the image to the specified file
with open(output_file, "wb") as f:
    f.write(image.data)

In [ ]:
input_shape = (96, 96, 3)
output_shape = 2

# Build the LeNet model and display its summary
model = build_custom_baseline(input_shape, output_shape)
model.summary()

In [ ]:
# Define the hyperparameters
batch_size = 16
epochs = 200

# Define callbacks
early_stopping = tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, mode='max', restore_best_weights=True)
lr_scheduler = tfk.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=5, factor=0.999, mode='max', min_lr=1e-5)

# Train the model and save its history
history = model.fit(
    x=X_train,
    y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, lr_scheduler]
).history

# Save the trained model
model.save('models/custom_baseline2')

In [ ]:
plot_training(history)

In [ ]:
plot_confusion_matrix('models/custom_baseline2', X_test, y_test)

####Transfer learning

##### 2 classes

In [ ]:
from tensorflow.keras.applications.efficientnet import preprocess_input

In [ ]:
efficient = tf.keras.applications.EfficientNetB0(
                    include_top=False,
                    weights="imagenet",
                    input_shape=(96, 96, 3),
                    pooling='avg',
                  )
#tfk.utils.plot_model(inceptionResnet, show_shapes=True)

efficient.trainable = False

# Define the augmentation
preprocessing = tfk.Sequential([
        tfkl.RandomFlip('horizontal_and_vertical'),
        tfkl.RandomRotation(0.3),
        tfkl.RandomContrast(0.1),
        #tfkl.RandomBrightness(0.001)
    ], name='Preprocessing')

# Create an input layer with shape (96, 96, 3)
inputs = tfk.Input(shape=(96, 96, 3))

# Data augmentation
augmentation = preprocessing(inputs)

# Connect MobileNetV2 to the input
x = efficient(augmentation)

x = tfkl.Dense(256, kernel_initializer=tfk.initializers.HeUniform(), activation='relu')(x)
x = tfkl.Dropout(1/12, seed=seed)(x)

# Add a Dense layer with 2 units and softmax activation as the classifier
outputs = tfkl.Dense(2, activation='softmax')(x)

# Create a Model connecting input and output
tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model with Categorical Cross-Entropy loss and Adam optimizer
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.AdamW(learning_rate=0.0005), metrics=['accuracy'])

# Display model summary
tl_model.summary()

In [ ]:
# Define callbacks
early_stopping = tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=16, mode='max', restore_best_weights=True)
lr_scheduler = tfk.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=5, factor=0.999, mode='max', min_lr=1e-5)


# Train the model
tl_history = tl_model.fit(
    x = preprocess_input(X_train*255),
    y = y_train,
    batch_size = 16,
    epochs = 100,
    class_weight={0: 0.806869961444094, 1: 1.3146773272415762},
    validation_data = (preprocess_input(X_val*255), y_val),
    callbacks = [early_stopping, lr_scheduler]
).history

In [ ]:
tl_model.save('models/transfer_learning_efficientnetb0_0')

In [ ]:
plot_training(tl_history)

In [ ]:
plot_confusion_matrix(preprocess_input(X_test*255), y_test, model_to_test=tl_model)

In [ ]:
del tl_model

##### 1 class

In [ ]:
from tensorflow.keras.applications.efficientnet import preprocess_input

In [ ]:
efficient = tf.keras.applications.EfficientNetB0(
                    include_top=False,
                    weights="imagenet",
                    input_shape=(96, 96, 3),
                    pooling='avg',
                  )

efficient.trainable = False


preprocessing = tfk.Sequential([
        tfkl.RandomFlip('horizontal_and_vertical'),
        #tfkl.RandomContrast(0.3),
        #tfkl.RandomBrightness(0.0005),
        #tfkl.ZeroPadding2D((2, 2)),
        #tfkl.RandomCrop(96, 96),
        tfkl.RandomRotation(0.5),
    ], name='Preprocessing')

classifier = tfk.Sequential([
      #tfkl.Dense(64, activation='relu'),
      #tfkl.Dropout(0.5),
      #tfkl.Dense(32, activation='relu'),
      #tfkl.Dropout(0.5),
      tfkl.Dense(256, activation='relu', kernel_initializer=tfk.initializers.HeUniform()),#4096
      tfkl.Dropout(0.5),
      tfkl.Dense(128, activation='relu', kernel_initializer=tfk.initializers.HeUniform()),#1024
      tfkl.Dropout(0.5),
      tfkl.Dense(1, activation='sigmoid')
], name='Classifier')

# Create an input layer with shape (96, 96, 3)
inputs = tfk.Input(shape=(96, 96, 3))

# Data augmentation
augmentation = preprocessing(inputs)

# Connect MobileNetV2 to the input
x = efficient(augmentation)

outputs = classifier(x)

tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

#tl_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
tl_model.compile(loss=tfk.losses.BinaryCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

tl_model.summary()


In [ ]:
# Define callbacks
early_stopping = tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=15, mode='max', restore_best_weights=True)
lr_scheduler = tfk.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=5, factor=0.999, mode='max', min_lr=1e-5)


# Train the model
tl_history = tl_model.fit(
    x = preprocess_input(X_train*255),
    y = y_train,
    batch_size = 16,
    epochs = 100,
    class_weight={0: 0.806869961444094, 1: 1.3146773272415762},
    validation_data = (preprocess_input(X_val*255), y_val),
    callbacks = [early_stopping, lr_scheduler]
).history

In [ ]:
tl_model.save('models/transfer_learning_efficientnetb0_0_sig')

In [ ]:
plot_training(tl_history)

In [ ]:
plot_confusion_matrix(preprocess_input(X_test*255), y_test, model_to_test=tl_model, one_class=True)

In [ ]:
del tl_model

#### Fine tuning

In [ ]:
# Re-load the model after transfer learning
ft_model = tfk.models.load_model('models/transfer_learning_efficientnetb0_4_sig')
ft_model.summary()

In [ ]:
# Set all MobileNetV2 layers as trainable
ft_model.get_layer('efficientnetb0').trainable = True
for i, layer in enumerate(ft_model.get_layer('efficientnetb0').layers):
   print(i, layer.name, layer.trainable)

In [ ]:
# Freeze first N layers, e.g., until the 133rd one
N = 222
for i, layer in enumerate(ft_model.get_layer('efficientnetb0').layers[:N]):
  layer.trainable=False
#for i, layer in enumerate(ft_model.get_layer('efficientnetb0').layers[207:]):
#  layer.trainable=False
for i, layer in enumerate(ft_model.get_layer('efficientnetb0').layers[N:]):
  if 'bn' in layer.name:
    layer.trainable=False
for i, layer in enumerate(ft_model.get_layer('efficientnetb0').layers):
   print(i, layer.name, layer.trainable)
ft_model.summary()

In [ ]:
# Compile the model
ft_model.compile(loss=tfk.losses.BinaryCrossentropy(), optimizer=tfk.optimizers.AdamW(1e-5), metrics='accuracy')

In [ ]:
early_stopping = tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=15, mode='max', restore_best_weights=True)
lr_scheduler = tfk.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=3, factor=0.999, mode='max', min_lr=1e-7)

# Fine-tune the model
ft_history = ft_model.fit(
    x = preprocess_input(new_X*255), # We need to apply the preprocessing thought for the MobileNetV2 network
    y = new_y,
    batch_size = 16,
    epochs = 200,
    validation_data = (preprocess_input(X_val*255), y_val), # We need to apply the preprocessing thought for the MobileNetV2 network
    callbacks = [early_stopping, lr_scheduler]
).history

In [ ]:
ft_model.save('models/ft_efficientnetb0_4_sig')

In [ ]:
del ft_model

In [ ]:
plot_confusion_matrix_sigmoid('models/ft_efficientnetb0_4_sig', preprocess_input(X_test*255), y_test)

### 2.4 Enrico's models

### Ensemble

In [ ]:
################################################################################
################################################################################
########## IT IS ENOUGH TO RUN THIS CELL TO BE READY TO THE TRAINING ###########
################################################################################
################################################################################
try:
    from google.colab import drive
    print("Code running on Google Colab... Conncecting to Google Drive...")
    drive.mount('/gdrive')
    %cd /gdrive/My Drive/[2023-2024] AN2DL/Homework 1
except:
    print("The code is not running on Google Colab...")

# Fix randomness and hide warnings
seed = 42

import os
import warnings
import logging
import random
import gc
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

random.seed(seed)

import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
#import shap
import cv2
np.random.seed(seed)

from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras.applications.mobilenet import preprocess_input
from tqdm import tqdm
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from IPython.display import Image
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)
print(f"Num GPUs Available: {len(tf.config.list_physical_devices('GPU'))}")

images_no_outliers = np.load('data/public_data_no_outliers.npy', allow_pickle=True)
labels_no_outliers = np.load('data/public_labels_no_outliers.npy', allow_pickle=True)

# Convert labels to one-hot encoding format
y = np.zeros((labels_no_outliers.shape[0], 2))
y[labels_no_outliers=='healthy', 0] = 1.
y[labels_no_outliers=='unhealthy', 1] = 1.

# Setting the input dataset to split
X = (images_no_outliers / 255).astype(np.float32)

# Defining the number of samples in the test set and in the validation set
validation_size = 750
test_size = 750

# Split data into train_val and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=seed, test_size=test_size, stratify=np.argmax(y,axis=1))

# Further split train_val into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=seed, test_size=validation_size, stratify=np.argmax(y_train_val,axis=1))

# Print shapes of the datasets
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

del(images_no_outliers)
del(labels_no_outliers)
gc.collect()

#### Ensemble class code

In [ ]:
from tensorflow.keras.models import clone_model
from sklearn.utils import resample
import gc
import time


class ModelEnsemble:
  """
  Class that represent an ensemble model

  Attributes
  ----------
  models: list(tensorflow.keras.Model)
    list of the models of the ensemble
  #histories: list(dict)
    list of the histories of training, if the models are trained by the class
    and not loaded
  path_to_models: str
    path where the models have to be stored or from where the models have to be
    loaded
  model_names_list: list(str)
    list of the names of the models in the ensemble
  num_models: int
      number of models to insert in the ensemble
  preprocessing_functions: list(funct)
    list of the preprocessing functions to apply to the inputs of the the models
  #X_train: numpy.ndarray
    images used as train set if the models are trained by the class and not
    loaded
  #y_train: numpy.ndarray
    labels of the images in the train set if the models are trained by the class
    and not loaded
  #X_val: numpy.ndarray
    images used as validation set if the models are trained by the class and not
    loaded
  #y_val: numpy.ndarray
    abels of the images in the train set if the models are trained by the class
    and not loaded
  #X_test: numpy.ndarray
    images used as test set if the models are trained by the class and not
    loaded
  #y_test: numpy.ndarray
    labels of the images in the test set if the models are trained by the class
    and not loaded

  """
  def __init__(self,
               model=None,
               num_models=2,
               preprocessing_input=None,
               X=np.array([]),
               y=np.array([]),
               validation_size=1000,
               test_size=1000,
               batch_size=16,
               epochs=10,
               loss=tfk.losses.CategoricalCrossentropy(),
               optimizer=tfk.optimizers.AdamW(),
               metrics=['accuracy'],
               callbacks=[tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, mode='max', restore_best_weights=True)],
               store_and_load=True,
               evaluate=True,
               ensemble_name='ensemble',
               model_names_list=[],
               preprocessing_functions=[],
               path_to_models='models/',
               seed=42,
               verbose=True):
    """
    Initializes the ensemble model

    Parameters
    ----------
    model: tensorflow.keras.Model
      model to be used to create the models of the ensemble, the training always
      starts from this model
    num_models: int
      number of models to insert in the ensemble
    preprocessing_input: funct
      preprocessing function to apply to the inputs of the neural network
    X: numpy.ndarray
      dataset of the images in input
    y: numpy.ndarray
      one-hot encoded labels associated to the images
    validation_size: int | float
      size in terms of samples or percentage of the validation set
    test_size: int | float
      size in terms of samples or percentage of the test set
    batch_size: int
      size of an input batch using in the training of the models
    epochs: int
      number of epochs of the training of the models
    loss: tensorflow.keras.losses.Loss
      loss to use during the training of the models
    optimizer: tensorflow.keras.optimisers.Optimizer
      optimizer to use during the training of the models
    metrics: list(str)
      list of the metrics to evaluate during the training
    callbacks: list(tensorflow.keras.callbacks.Callback)
      callbacks to run during the training of the models
    store_and_load: bool
      if True, the models of the ensemble are stored and deleted after training
      in order to save space in the memory; they are all loaded again after the
      training of last model has ended
    evaluate: bool
      if True, the ensemble is evaluated on the test set after the training, if
      the models are trained by the class and not loaded
    ensemble_name: str
      name to give to the ensemble model, the models are stored using the
      pattern: path_to_models + ensemble_name + index of the trained model
    model_names_list: list(str)
      list of the names of the models in the ensemble
    preprocessing_functions: list(funct)
      list of the preprocessing functions to apply to the inputs of the the
      models
    path_to_models: str
      path where the models have to be stored or from where the models have to be
      loaded
    seed: int
      seed to use to allow reproducibility
    verbose: bool
      if True, textual information about the operations performed by the object
      are provided

    """

    self.models = []
    #self.histories = []
    self.path_to_models = path_to_models
    self.model_names_list = []

    if model is None:
      # Storing the names of the models in the ensemble
      self.model_names_list = model_names_list
      self.num_models = len(model_names_list)
      self.preprocessing_functions = preprocessing_functions
    else:
      self.num_models = num_models
      self.preprocessing_functions = [preprocessing_input for i in range(num_models)]
      self.train_ensemble(model, num_models, preprocessing_input, X, y,
                          validation_size, test_size, batch_size, epochs, loss,
                          optimizer, metrics, callbacks, store_and_load,
                          evaluate, ensemble_name, path_to_models, seed, verbose)

  def train_ensemble(self,
                     model,
                     num_models,
                     preprocessing_input,
                     X,
                     y,
                     validation_size,
                     test_size,
                     batch_size,
                     epochs,
                     loss,
                     optimizer,
                     metrics,
                     callbacks,
                     store_and_load,
                     evaluate,
                     ensemble_name,
                     path_to_models,
                     seed,
                     verbose):

    """
    Trains the ensemble model

    Parameters
    ----------
    model: tensorflow.keras.Model
      model to be used to create the models of the ensemble, the training always
      starts from this model
    num_models: int
      number of models to insert in the ensemble
    preprocessing_input: funct
      preprocessing function to apply to the inputs of the neural network
    X: numpy.ndarray
      dataset of the images in input
    y: numpy.ndarray
      one-hot encoded labels associated to the images
    validation_size: int | float
      size in terms of samples or percentage of the validation set
    test_size: int | float
      size in terms of samples or percentage of the test set
    batch_size: int
      size of an input batch using in the training of the models
    epochs: int
      number of epochs of the training of the models
    loss: tensorflow.keras.losses.Loss
      loss to use during the training of the models
    optimizer: tensorflow.keras.optimisers.Optimizer
      optimizer to use during the training of the models
    metrics: list(str)
      list of the metrics to evaluate during the training
    callbacks: list(tensorflow.keras.callbacks.Callback)
      callbacks to run during the training of the models
    store_and_load: bool
      if True, the models of the ensemble are stored and deleted after training
      in order to save space in the memory; they are all loaded again after the
      training of last model has ended
    evaluate: bool
      if True, the ensemble is evaluated on the test set after the training, if
      the models are trained by the class and not loaded
    ensemble_name: str
      name to give to the ensemble model, the models are stored using the
      pattern: path_to_models + ensemble_name + index of the trained model
    path_to_models: str
      path where the models have to be stored or from where the models have to be
      loaded
    seed: int
      seed to use to allow reproducibility
    verbose: bool
      if True, textual information about the operations performed by the object
      are provided

    """

    # Record the start time
    start_time = time.time()

    # Split data into train_val and test sets
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=seed, test_size=test_size, stratify=np.argmax(y,axis=1))

    # Further split train_val into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=seed, test_size=validation_size, stratify=np.argmax(y_train_val,axis=1))

    # Set the sets in the object
    #self.X_test = X_test
    #self.y_test = y_test
    #self.X_val = X_val
    #self.y_val = y_val
    #self.X_train = X_train
    #self.y_train = y_train

    # Print shapes of the datasets
    if verbose:
      print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
      print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
      print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")
      print()

    # Training the models of the ensemble
    for i in range(num_models):
      # Cloning the model to train (the weights are randomly initialized)
      new_model = clone_model(model)
      # Copying the weights
      new_model.set_weights(model.get_weights())

      # Create a new instance of the optimizer using the configuration
      new_optimizer = type(optimizer).from_config(optimizer.get_config())
      #new_optimizer=tfk.optimizers.AdamW()

      # Compiling the model
      new_model.compile(loss=loss, optimizer=new_optimizer, metrics=metrics)

      # Displaying the summary of the model
      if verbose:
        new_model.summary()

      if verbose:
        print(f'The learning of the model {i} started')
        print()

      # Bootstrapping the dataset
      indexes = resample(range(X_train.shape[0]), random_state=seed, stratify=y_train)
      new_X_train = X_train[indexes]
      new_y_train = y_train[indexes]

      # Running the garbage collector
      gc.collect()

      # Training a model
      ft_history = new_model.fit(
          x = self.preprocessing_functions[i](new_X_train*255),
          y = new_y_train,
          batch_size = batch_size,
          epochs = epochs,
          validation_data = (self.preprocessing_functions[i](X_val*255), y_val),
          callbacks = callbacks
      ).history

      if store_and_load:
        if verbose:
          print(f'Storing model {ensemble_name}{i} started into {path_to_models}')
          print()

        # Storing the model, if requested, in order to save space
        new_model.save(f'{path_to_models}{ensemble_name}{i}')
        # Adding the name of the model to the list of the names of the models
        self.model_names_list.append(f'{ensemble_name}{i}')

        # Deleting the model
        del new_model
      else:
        self.models.append(new_model)

      #self.histories.append(ft_history)

    # Loading the models if they were stored and deleted during the training
    #if store_and_load:
    #  for i in range(num_models):
    #    self.models.append(tfk.models.load_model(f'{path_to_models}{ensemble_name}{i}'))

    # Recording the end time
    end_time = time.time()
    # Calculating and printing the elapsed time
    if verbose:
      print(f"Elapsed time: {(end_time - start_time)/3600} hours")

    # Evaluating the model
    if evaluate:
      self.evaluate()

  def get_models(self):
    """
    Returns the models that compose the ensemble

    Returns
    -------
    list(tensorflow.keras.Model)
      list of models that compose the ensemble

    """

    return self.models

  def get_histories(self):
    """
    Returns the histories of the train of the models composing the ensemble

    Returns
    -------
    dict
      list of histories of the train of the models composing the ensemble

    """

    return self.histories


  def predict_one_model(self, X, model_name, i, verbose=True):
    """
    Predicts the probability distribution of the labels of the given inputs for
    the i-th model of the ensemble

    Parameters
    ----------
    X: numpy.ndarray
      input images on which the prediction has to be computed
    model_name: str
      name of the model in to use to compute the predictions
    i: int
      index of the model in the ensemble to use to compute the predictions
    verbose: bool
      if True, textual information about the operations performed by the object
      are provided

    Returns
    -------
    list(numpy.ndarray)
      list of the predicted probability distribution of the labels

    """

    if verbose:
        print(f'Loading the model {model_name} ...')
    model = tfk.models.load_model(self.path_to_models+model_name)
    y_pred = model.predict(self.preprocessing_functions[i](X))

    # Deleting the model
    del model
    # Running the garbage collector
    gc.collect()

    return y_pred


  def predict(self, X, mode='conf_weighted', weight_models=False, prob=False, verbose=True):
    """
    Predicts the labels of the input images

    Parameters
    ----------
    X: numpy.ndarray
      input images on which the prediction has to be computed
    mode: str
      'conf_weighted', if the prediction has to be done considering the confidence
      that the models have in predicting a class, 'majority', if the prediction
      has to be done using majority voting
    #weight_models: bool

    prob: bool
      if True, the probabilities of the positive class are returned instead of
      the actual predicted class
    verbose: bool
      if True, textual information about the operations performed by the object
      are provided

    Returns
    -------
    numpy.ndarray
      predicted labels of the input images | probabilities of the positive class

    """

    # TODO we can try to change the way we compute the final labels and we weight
    # the models

    # Predicting the probability distribution
    y_pred = np.array([])
    if mode == 'conf_weighted':
      y_pred = self.predict_one_model(X, self.model_names_list[0], 0)
      for i,model_name in enumerate(self.model_names_list[1:]):
        y_pred += self.predict_one_model(X, model_name, 0)
        # Running the garbage collector
        gc.collect()
      y_pred = y_pred[:,1]/len(self.model_names_list) if prob else tf.argmax(y_pred, axis=-1).numpy()
    elif mode == 'majority':
      y_pred = tf.round(self.predict_one_model(X, self.model_names_list[0], 0)).numpy()
      for i,model_name in enumerate(self.model_names_list[1:]):
        y_pred += tf.round(self.predict_one_model(X, model_name, i)).numpy()
        # Running the garbage collector
        gc.collect()
      y_pred = y_pred[:,1]/len(self.model_names_list) if prob else tf.argmax(y_pred, axis=-1).numpy()

    return y_pred

  def evaluate(self):
    """
    Evaluates the ensemble model on the test set

    """

    # Computing the predictions
    predictions = self.predict(self.X_test*255)

    # Creating the confusion matrix
    cm = confusion_matrix(np.argmax(self.y_test, axis=-1), predictions)

    # Computing the classification metrics
    accuracy = accuracy_score(np.argmax(self.y_test, axis=-1), predictions)
    precision = precision_score(np.argmax(self.y_test, axis=-1), predictions)
    recall = recall_score(np.argmax(self.y_test, axis=-1), predictions)
    f1 = f1_score(np.argmax(self.y_test, axis=-1), predictions)

    # Displaying the computed metrics
    print('Accuracy:', accuracy.round(4))
    print('Precision:', precision.round(4))
    print('Recall:', recall.round(4))
    print('F1:', f1.round(4))

    # Plotting the confusion matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm.T, xticklabels=list(('healthy','unhealthy')), yticklabels=list(('healthy','unhealthy')), cmap='Blues', annot=True)
    plt.xlabel('True labels')
    plt.ylabel('Predicted labels')
    plt.show()


#### Models to use in the ensemble

##### EfficientnetB0

In [ ]:
from tensorflow.keras.applications.efficientnet import preprocess_input as preprocess_input_efficientnet

efficientnet = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(96, 96, 3),
    pooling="avg",

)

#tfk.utils.plot_model(efficientnet, show_shapes=True)

In [ ]:
# Use the supernet as feature extractor, i.e. freeze all its weigths
efficientnet.trainable = False

# Create an input layer with shape (96, 96, 3)
inputs = tfk.Input(shape=(96, 96, 3))

# Defining and evaluating the augmentation layer
preprocessing = tfk.Sequential([
      tfkl.RandomFlip('horizontal', name='RandomFlip_horizontal'),
      tfkl.RandomFlip('vertical', name='RandomFlip_vertical'),
      tfkl.RandomRotation(0.3, name='RandomRotation'),
      tfkl.RandomContrast(0.3),
  ], name='Preprocessing')


preprocessed = preprocessing(inputs)

# Connect to the input
x = efficientnet(preprocessed)

# Adding layers for the classification
dropout_rate = 1/12

dense = tfkl.Dense(128, activation='relu', kernel_initializer=tfk.initializers.HeUniform())(x)
dropout = tfkl.Dropout(dropout_rate, seed=seed)(dense)

# Add a Dense layer with 2 units and softmax activation as the classifier
outputs = tfkl.Dense(2, activation='softmax')(dropout)

# Create a Model connecting input and output
tf_efficient_model = tfk.Model(inputs=inputs, outputs=outputs, name='elementar_mode_ensemble')

# Compile the model with Categorical Cross-Entropy loss and Adam optimizer
#tf_efficient_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.AdamW(), metrics=['accuracy'])

# Display model summary
tf_efficient_model.summary()

In [ ]:
ensemble = ModelEnsemble(model=tf_efficient_model,
                         preprocessing_input=preprocess_input_efficientnet,
                         num_models=50,
                         X=X,
                         y=y,
                         validation_size=100,
                         test_size=500,
                         ensemble_name='ensemble_2_')

##### Custom model

In [ ]:
# Define a convolutional block function with multiple parameters
def conv_block_1(x, filters, kernel_size, padding='same', downsample=True, activation='relu', stack=2, name=''):

  # If downsample is True, apply max-pooling
  if downsample:
    x = tfkl.MaxPooling2D(name='MaxPool_' + name)(x)

  # Apply a stack of convolutional layers with specified filters, kernel size, and activation
  for s in range(stack):
    x = tfkl.Conv2D(filters=filters, kernel_size=kernel_size, padding=padding, name='Conv_' + name + str(s+1))(x)
    x = tfkl.Activation(activation, name='Activation_' + name + str(s+1))(x)

  return x

# Define the model
def build_custom_baseline_augmentation(input_shape=(96, 96, 3), output_shape=2, learning_rate=0.001, seed=42):

  # Input layer
  input_layer = tfkl.Input(shape=input_shape, name='Input_Layer')

  # Define a preprocessing Sequential model with random flip, zero padding, and random crop
  preprocessing = tfk.Sequential([
        tfkl.RandomFlip('horizontal', name='RandomFlip_horizontal'),
        tfkl.RandomFlip('vertical', name='RandomFlip_vertical'),
        tfkl.RandomRotation(0.3, name='RandomRotation'),
        #tfkl.RandomBrightness(0.001, name='RandomBrigtness'),
        tfkl.RandomContrast(0.3),
    ], name='Preprocessing')

  # Apply preprocessing to input layer
  x0 = preprocessing(input_layer)

  # Create convolutional blocks
  x1 = conv_block_1(x=x0, filters=32, kernel_size=3, downsample=False, stack=1, name='1')
  x2 = conv_block_1(x=x1, filters=32, kernel_size=3, downsample=True, stack=1, name='2')
  x3 = conv_block_1(x=x2, filters=64, kernel_size=3, downsample=True, stack=1, name='3')
  x4 = conv_block_1(x=x3, filters=64, kernel_size=3, downsample=True, stack=1, name='4')
  x5 = conv_block_1(x=x4, filters=128, kernel_size=3, downsample=True, stack=1, name='5')
  dropout_rate = 1/6
  dropout = tfkl.Dropout(dropout_rate, seed=seed)(x5)

  # Global Average Pooling and classifier
  x = tfkl.GlobalAveragePooling2D(name='GlobalAveragePooling')(dropout)
  x = tfkl.Dense(output_shape, name='Dense')(x)
  output_activation = tfkl.Activation('softmax', name='Softmax')(x)

  # Create the model
  model = tfk.Model(inputs=input_layer, outputs=output_activation, name='baseline_augmentation')

  # Define optimizer, loss, and metrics
  optimizer = tfk.optimizers.AdamW(learning_rate, weight_decay=5e-4)
  loss = tfk.losses.CategoricalCrossentropy()
  metrics = ['accuracy']

  # Compile the model
  model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
  return model

input_shape = (96, 96, 3)
output_shape = 2

# Build the LeNet model and display its summary
custom_model = build_custom_baseline_augmentation(input_shape, output_shape, learning_rate=0.001)
custom_model.summary()

In [ ]:
def preprocess_costom(X):
  return X/255
ensemble = ModelEnsemble(model=custom_model,
                         preprocessing_input=preprocess_costom,
                         num_models=50,
                         X=X,
                         y=y,
                         validation_size=100,
                         test_size=500,
                         ensemble_name='ensemble_custom_model_01_')

##### Custom model complex

In [ ]:
# Define a residual convolutional block with optional batch normalization
def conv_residual_block(x, filters, kernel_size, padding='same', downsample=True, activation='relu', stack=2, batch_norm=True, name=''):

    # If downsample is True, apply max-pooling
    if downsample:
        x = tfkl.MaxPooling2D(name='MaxPool_' + name)(x)

    # Create a copy of the input for the residual connection
    x_ = x

    # Apply a stack of convolutional layers to the copy
    for s in range(stack):
        x_ = tfkl.Conv2D(filters=filters, kernel_size=kernel_size, padding=padding, name='Conv_' + name + str(s+1))(x_)
        if batch_norm:
            x_ = tfkl.BatchNormalization(name='BatchNorm_' + name + str(s+1))(x_)
        x_ = tfkl.Activation(activation, name='Activation_' + name + str(s+1))(x_)

    # If downsample is True, apply a 1x1 convolution to match the number of filters
    if downsample:
        x = tfkl.Conv2D(filters=filters, kernel_size=1, padding=padding, name='Conv_' + name + 'skip')(x)

    # Add the original and the processed copy to create the residual connection
    x = tfkl.Add(name='Add_' + name)([x_, x])

    return x

# Define a function to build a VGG18-like model with residual blocks
def build_model(input_shape=(96, 96, 3), output_shape=2, learning_rate=1e-4, seed=seed):

    # Input layer
    input_layer = tfkl.Input(shape=input_shape, name='Input_Layer')

    # Define a preprocessing Sequential model with random flip, zero padding, and random crop
    preprocessing = tfk.Sequential([
        tfkl.RandomFlip('horizontal', name='RandomFlip_horizontal'),
        tfkl.ZeroPadding2D((2, 2), name='ZeroPadding_2x2'),
        tfkl.RandomCrop(input_shape[0], input_shape[1], name='RandomCrop')
    ], name='Preprocessing')

    # Apply preprocessing to the input layer
    x0 = preprocessing(input_layer)

    # Initial convolution with batch normalization and activation
    x0 = tfkl.Conv2D(filters=64, kernel_size=3, padding='same', name='Conv0')(x0)
    x0 = tfkl.BatchNormalization(name='BatchNorm0')(x0)
    x0 = tfkl.Activation('relu', name='ReLU0')(x0)

    # Create residual blocks
    x1 = conv_residual_block(x=x0, filters=64, kernel_size=3, downsample=False, stack=2, name='1')
    x1 = conv_residual_block(x=x1, filters=64, kernel_size=3, downsample=False, stack=2, name='2')

    x2 = conv_residual_block(x=x1, filters=128, kernel_size=3, downsample=True, stack=2, name='3')
    x2 = conv_residual_block(x=x2, filters=128, kernel_size=3, downsample=False, stack=2, name='4')

    x3 = conv_residual_block(x=x2, filters=256, kernel_size=3, downsample=True, stack=2, name='5')
    x3 = conv_residual_block(x=x3, filters=256, kernel_size=3, downsample=False, stack=2, name='6')

    x4 = conv_residual_block(x=x3, filters=512, kernel_size=3, downsample=True, stack=2, name='7')
    x4 = conv_residual_block(x=x4, filters=512, kernel_size=3, downsample=False, stack=2, name='8')

    # Global Average Pooling and classifier
    x = tfkl.GlobalAveragePooling2D(name='GlobalAveragePooling')(x4)
    x = tfkl.Dense(output_shape, name='Dense')(x)
    output_activation = tfkl.Activation('softmax', name='Softmax')(x)

    # Create the model
    model = tfk.Model(inputs=input_layer, outputs=output_activation, name='VGG18_Residual')

    # Define optimizer, loss, and metrics
    # AdamW is an Adam optimizer which applies weight_decay to network layers,
    # i.e it's another way to apply l2 regularization to the whole network
    optimizer = tfk.optimizers.AdamW(learning_rate, weight_decay=5e-4)
    loss = tfk.losses.CategoricalCrossentropy()
    metrics = ['accuracy']

    # Compile the model
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    return model

custom_model = build_model()
#residual_model.summary()
#tfk.utils.plot_model(residual_model, expand_nested=True, show_shapes=True)

In [ ]:
def preprocess_costom(X):
  return X/255
ensemble = ModelEnsemble(model=custom_model,
                         preprocessing_input=preprocess_costom,
                         num_models=50,
                         X=X,
                         y=y,
                         validation_size=100,
                         test_size=500,
                         ensemble_name='ensemble_custom_model_01_')

### Testing Ensemble through loading

In [ ]:
################################################################################
################################################################################
################################################################################
################################################################################
# Altro modo per caricare i modelli e fare l'ensemble, non serve runnare questa
# cella
################################################################################
################################################################################
################################################################################
################################################################################



# It is possible also to ensemble different models (it could be risky)
from tensorflow.keras.applications.efficientnet import preprocess_input as preprocess_input_efficientnet

def preprocess_costom(X):
  return X/255

path_to_models = 'models/'
model_names = []
num_models = 44
for i in range(num_models):
  model_names.append(f'ensemble_custom_model_01_{i}')

preprocessing_functions = [preprocess_costom for i in range(num_models)]

loaded_ensemble = ModelEnsemble(model_names_list=model_names,
                                preprocessing_functions=preprocessing_functions,
                                path_to_models=path_to_models)

In [ ]:
modes = ['conf_weighted', 'majority']
seed = 42
validation_size=100
test_size=500

# Split data into train_val and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=seed, test_size=test_size, stratify=np.argmax(y,axis=1))

# Further split train_val into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=seed, test_size=validation_size, stratify=np.argmax(y_train_val,axis=1))

del(X)
del(y)

# Running the garbage collector
gc.collect()

for mode in modes:

  # Computing the predictions
  predictions = loaded_ensemble.predict(X_test*255, mode=mode)

  # Creating the confusion matrix
  cm = confusion_matrix(np.argmax(y_test, axis=-1), predictions)

  # Computing the classification metrics
  accuracy = accuracy_score(np.argmax(y_test, axis=-1), predictions)
  precision = precision_score(np.argmax(y_test, axis=-1), predictions)
  recall = recall_score(np.argmax(y_test, axis=-1), predictions)
  f1 = f1_score(np.argmax(y_test, axis=-1), predictions)

  # Displaying the computed metrics
  print('Accuracy:', accuracy.round(4))
  print('Precision:', precision.round(4))
  print('Recall:', recall.round(4))
  print('F1:', f1.round(4))

  # Plotting the confusion matrix
  plt.figure(figsize=(10, 8))
  sns.heatmap(cm.T, xticklabels=list(('healthy','unhealthy')), yticklabels=list(('healthy','unhealthy')), cmap='Blues', annot=True)
  plt.xlabel('True labels')
  plt.ylabel('Predicted labels')
  plt.show()

  # Running the garbage collector
  gc.collect()

### For the submissions

In [ ]:
# TO BE USED IN THE SUBMITS

import os
import numpy
import tensorflow as tf
from tensorflow.keras.applications.efficientnet import preprocess_input as preprocess_input_efficientnet

class model:
    def __init__(self, path):
        self.models = []
        model_names_list = ['ensemble0', 'ensemble1', 'ensemble2', 'ensemble3', 'ensemble4', 'ensemble5', 'ensemble6', 'ensemble7', 'ensemble8']
        for model_name in model_names_list:
            self.models.append(tf.keras.models.load_model('models/'+model_name))#os.path.join(path, model_name)))

    def predict(self, X):
        out = self.predict2(preprocess_input_efficientnet(X))
        return out

    def predict_prob(self, X):
        y_pred = []
        for model in self.models:
            y_pred.append(model.predict(X))

        return y_pred

    def predict2(self, X, prob=False):
        y_pred_prob = self.predict_prob(X)
        y_pred_models = []
        for pred in y_pred_prob:
            y_pred_models.append(tf.argmax(pred, axis=-1))
        y_pred = numpy.array(y_pred_models)

        return tf.convert_to_tensor(numpy.mean(y_pred, axis=0)) if prob else tf.convert_to_tensor(numpy.round(numpy.mean(y_pred, axis=0)))

In [ ]:
model1 = model('path')

In [ ]:
# Computing the predictions
predictions = model1.predict(X_test*255)

# Creating the confusion matrix
cm = confusion_matrix(np.argmax(y_test, axis=-1), predictions)

# Computing the classification metrics
accuracy = accuracy_score(np.argmax(y_test, axis=-1), predictions)
precision = precision_score(np.argmax(y_test, axis=-1), predictions)
recall = recall_score(np.argmax(y_test, axis=-1), predictions)
f1 = f1_score(np.argmax(y_test, axis=-1), predictions)

# Displaying the computed metrics
print('Accuracy:', accuracy.round(4))
print('Precision:', precision.round(4))
print('Recall:', recall.round(4))
print('F1:', f1.round(4))

# Plotting the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm.T, xticklabels=list(('healthy','unhealthy')), yticklabels=list(('healthy','unhealthy')), cmap='Blues', annot=True)
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

## 3. Results and conclusions



In [ ]:
# Find the epoch with the highest validation accuracy
best_epoch = np.argmax(history['val_accuracy'])

# Plot training and validation performance metrics
plt.figure(figsize=(20, 5))

# Plot training and validation loss
plt.plot(history['loss'], label='Training', alpha=0.8, color='#ff7f0e', linewidth=3)
plt.plot(history['val_loss'], label='Validation', alpha=0.8, color='#4D61E2', linewidth=3)
plt.legend(loc='upper left')
plt.title('Binary Crossentropy')
plt.grid(alpha=0.3)

plt.figure(figsize=(20, 5))

# Plot training and validation accuracy, highlighting the best epoch
plt.plot(history['accuracy'], label='Training', alpha=0.8, color='#ff7f0e', linewidth=3)
plt.plot(history['val_accuracy'], label='Validation', alpha=0.8, color='#4D61E2', linewidth=3)
plt.plot(best_epoch, history['val_accuracy'][best_epoch], marker='*', alpha=0.8, markersize=10, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=0.3)

plt.show()

# There is no overfitting -> good!

In [ ]:
pred = model.predict(np.expand_dims(images[0], axis=0), verbose=0)
pred, labels[0]

## Train MobileNetV2 from scratch

In [ ]:
# Create MobileNetV2 model with specified settings
mobile = tfk.applications.MobileNetV2(
    input_shape=(96, 96, 3),
    include_top=False,  # We avoid to include the fully connected layers that
                        # are designed for a 1000-classes classification.
    weights=None,       # We will not download the trained weights, we'll have
                        # only the network (we train from skratch).
    pooling='avg',      # Type of pooling.
)

# Display the model architecture with input shapes
tfk.utils.plot_model(mobile, show_shapes=True)

# DepthwiseConv2D is another different type of layer.

# There are skip connections. The most important information are at the lowest
# levels, in sequential networks I can extract information from the last layer,
# adding skip connections, I'm able to exploit information form the both the
# initial and final levels.

# Using skip connections, I need to merge the information coming from different
# layers in the correct way: in case of concatenation, I have to concatenate
# kernels, in case of addition, I have need to have exactly the same shape of
# the activations.

In [ ]:
# Create an input layer with shape (96, 96, 3)
inputs = tfk.Input(shape=(96, 96, 3))
# Connect MobileNetV2 to the input
x = mobile(inputs)
# Add a Dense layer with 2 units and softmax activation as the classifier
outputs = tfkl.Dense(2, activation='softmax')(x)

# Create a Model connecting input and output
model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model with Categorical Cross-Entropy loss and Adam optimizer
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics=['accuracy'])

# Display model summary
model.summary()

In [ ]:
# Train the model
history = model.fit(
    x = preprocess_input(X_train*255), # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train,
    batch_size = 16,
    epochs = 200,
    validation_data = (preprocess_input(X_val*255), y_val), # We need to apply the preprocessing thought for the MobileNetV2 network
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True)]
).history

# Here we are training from skretch the network (this is not transfer learning).
# We are overfitting a lot, the network is completely unable to generalise.
# We can see that the accuracy on the training set is almost 100% and the
# accuracy on the validation set is 50% (random guess).
# The network is not able to generalise since training such a big network is not
# easy and needs some steps to do and some particular techniques. Moreover here
# we have a too small input dataset.

# To train big networks we need to apply particular pipelines of learning rates,
# we need to apply warmup (training using small learning rates).

In [ ]:
# Plot the re-trained MobileNetV2 training history
plt.figure(figsize=(15,5))
plt.plot(history['loss'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(history['val_loss'], label='Re-trained', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(history['val_accuracy'], label='Re-trained', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

## Transfer Learning
Leveraging pre-trained network for a different classification problem

In [ ]:
mobile = tfk.applications.MobileNetV2(
    input_shape=(96, 96, 3),
    include_top=False,
    weights="imagenet",
    pooling='avg',
)
tfk.utils.plot_model(mobile, show_shapes=True)

# Transfer learning: we want to freeze the parameters of the feature extraction
# network and we want to train the classification head (fully connected NN).

In [ ]:
# Use the supernet as feature extractor, i.e. freeze all its weigths
mobile.trainable = False

# Create an input layer with shape (224, 224, 3)
inputs = tfk.Input(shape=(96, 96, 3))
# Connect MobileNetV2 to the input
x = mobile(inputs)
# Add a Dense layer with 2 units and softmax activation as the classifier
outputs = tfkl.Dense(2, activation='softmax')(x)

# Create a Model connecting input and output
tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model with Categorical Cross-Entropy loss and Adam optimizer
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics=['accuracy'])

# Display model summary
tl_model.summary()

In [ ]:
# Train the model
tl_history = tl_model.fit(
    x = preprocess_input(X_train*255), # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train,
    batch_size = 16,
    epochs = 200,
    validation_data = (preprocess_input(X_val*255), y_val), # We need to apply the preprocessing thought for the MobileNetV2 network
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

In [ ]:
# Evaluate the model on the test set
test_accuracy = tl_model.evaluate(preprocess_input(X_test*255),y_test,verbose=0)[-1]
print('Test set accuracy %.4f' % test_accuracy)

In [ ]:
# Plot the re-trained and the transfer learning MobileNetV2 training histories
plt.figure(figsize=(15,5))
#plt.plot(history['loss'], alpha=.3, color='#ff7f0e', linestyle='--')
#plt.plot(history['val_loss'], label='Re-trained', alpha=.8, color='#ff7f0e')
plt.plot(tl_history['loss'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(tl_history['val_loss'], label='Transfer Learning', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
#plt.plot(history['accuracy'], alpha=.3, color='#ff7f0e', linestyle='--')
#plt.plot(history['val_accuracy'], label='Re-trained', alpha=.8, color='#ff7f0e')
plt.plot(tl_history['accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(tl_history['val_accuracy'], label='Transfer Learning', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

# We achieved the performance of the network we designed in a small number of
# epochs.

In [ ]:
# Predict labels for the entire test set
predictions = tl_model.predict(preprocess_input(X_test*255), verbose=0)

# Display the shape of the predictions
print("Predictions Shape:", predictions.shape)

# Compute the confusion matrix
cm = confusion_matrix(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1))

# Compute classification metrics
accuracy = accuracy_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1))
precision = precision_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')
recall = recall_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')
f1 = f1_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')

# Display the computed metrics
print('Accuracy:', accuracy.round(4))
print('Precision:', precision.round(4))
print('Recall:', recall.round(4))
print('F1:', f1.round(4))

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm.T, xticklabels=list(('healthy','unhealthy')), yticklabels=list(('healthy','unhealthy')), cmap='Blues', annot=True)
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

# Still good performances

In [ ]:
# Save the best model
tl_model.save('TransferLearningModel_2')
#del tl_model

In [ ]:
m = tf.keras.models.load_model('trial')

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications.mobilenet import preprocess_input

class model:
    def __init__(self, path):
        self.model = tf.keras.models.load_model( 'trial')

    def predict(self, X):

        # Note: this is just an example.
        # Here the model.predict is called, followed by the argmax
        out = self.model.predict(preprocess_input(X*255))
        out = tf.argmax(out, axis=-1)  # Shape [BS]

        return out

In [ ]:
model = model(os.getcwd())
pred = model.predict(X_test)
pred

In [ ]:
np.argmax(predictions, axis=-1)

## Fine Tuning
Training also the backbone network. To this purpose it is better to
1.   Perform transfer learning, namely train the new classification head only;
2.   Unfreeze the backbone layers (or possibly a few of them depending on the amount of training data) and train the entire netrwork. Note that the classification head needs to be a properly trained model. We adjust the last set of transformations, we use a very small learning rate.

Training a neural network cannot be done end-to-end. Most effective trainings are split in many steps.



In [ ]:
# Re-load the model after transfer learning
ft_model = tfk.models.load_model('TransferLearningModel_1')
ft_model.summary()

In [ ]:
# Set all MobileNetV2 layers as trainable
ft_model.get_layer('mobilenetv2_1.00_96').trainable = True
for i, layer in enumerate(ft_model.get_layer('mobilenetv2_1.00_96').layers):
   print(i, layer.name, layer.trainable)

# I can also iteratively unfreeze only some parts of the network and do the
# training, e.g. the first time, only the last layer; the second time, only the
# last two, and so on.
# We can also unfreeze different parts of the networks in different steps of the
# training keeping freezed the others, training alternatively different parts of
# the network. We can change the learning rates training different parts (e.g.
# higher learning rates on the last layers and smaller learning rates in the
# case of the training of the CNN layers).

In [ ]:
# Freeze first N layers, e.g., until the 133rd one
N = 133
for i, layer in enumerate(ft_model.get_layer('mobilenetv2_1.00_96').layers[:N]):
  layer.trainable=False
for i, layer in enumerate(ft_model.get_layer('mobilenetv2_1.00_96').layers):
   print(i, layer.name, layer.trainable)
ft_model.summary()

In [ ]:
# Compile the model
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-5), metrics='accuracy')

In [ ]:
# Fine-tune the model
ft_history = ft_model.fit(
    x = preprocess_input(X_train*255), # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train,
    batch_size = 16,
    epochs = 200,
    validation_data = (preprocess_input(X_val*255), y_val), # We need to apply the preprocessing thought for the MobileNetV2 network
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True)]
).history

# The first epoch is already more performing than all the previous model we have
# created.

In [ ]:
# Plot the re-trained, the transfer learning and the fine-tuned MobileNetV2 training histories
plt.figure(figsize=(15,5))
plt.plot(history['loss'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(history['val_loss'], label='Re-trained', alpha=.8, color='#ff7f0e')
plt.plot(tl_history['loss'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(tl_history['val_loss'], label='Transfer Learning', alpha=.8, color='#4D61E2')
plt.plot(ft_history['loss'], alpha=.3, color='#408537', linestyle='--')
plt.plot(ft_history['val_loss'], label='Fine Tuning', alpha=.8, color='#408537')
plt.legend(loc='upper left')
plt.title('Binary Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(history['val_accuracy'], label='Re-trained', alpha=.8, color='#ff7f0e')
plt.plot(tl_history['accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(tl_history['val_accuracy'], label='Transfer Learning', alpha=.8, color='#4D61E2')
plt.plot(ft_history['accuracy'], alpha=.3, color='#408537', linestyle='--')
plt.plot(ft_history['val_accuracy'], label='Fine Tuning', alpha=.8, color='#408537')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

# We can also skip the transfer learning step but it will be more difficult for
# the network to learn.

In [ ]:
# Evaluate the model on the test set
test_accuracy = ft_model.evaluate(preprocess_input(X_test*255),y_test,verbose=0)[-1]
print('Test set accuracy %.4f' % test_accuracy)

# Very good performance on the test set.

In [ ]:
# Predict labels for the entire test set
predictions = ft_model.predict(preprocess_input(X_test*255), verbose=0)

# Display the shape of the predictions
print("Predictions Shape:", predictions.shape)

# Compute the confusion matrix
cm = confusion_matrix(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1))

# Compute classification metrics
accuracy = accuracy_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1))
precision = precision_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')
recall = recall_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')
f1 = f1_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')

# Display the computed metrics
print('Accuracy:', accuracy.round(4))
print('Precision:', precision.round(4))
print('Recall:', recall.round(4))
print('F1:', f1.round(4))

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm.T, xticklabels=list(('healthy','unhealthy')), yticklabels=list(('healthy','unhealthy')), cmap='Blues', annot=True)
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

# Still good performances

In [ ]:
# Save the best model
ft_model.save('FineTuningModel')
del ft_model

### Test on your own data

In [ ]:
# Re-load the model after transfer learning
ft_model = tfk.models.load_model('FineTuningModel')
ft_model.summary()

In [ ]:
test_path = 'test_images/'
test = load_images_from_folder(test_path)
test.shape

In [ ]:
# predict the test image
test_predictions = ft_model.predict(preprocess_input(test*255), verbose=0)

index = 2
prediction = test_predictions[index]

if prediction[0] >= prediction[1]:
    label = 'item'
    probability = float("{:.{}f}".format(prediction[0], 4))*100

else:
    label = 'animal'
    probability = float("{:.{}f}".format(prediction[1], 4))*100

plt.title(f'{label} ({probability}%)')
plt.imshow(np.squeeze(test[index]))
plt.show()

### TODO Exercise
Practice the various transformations, create your own successful preprocessing pipeline and compare different Keras applications' architectures.

What happens if you combine augmentation techniques with fine tuning?

*Credits: Eugenio Lomurno, 📧 eugenio.lomurno@polimi.it*